In [1]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import math
import controls as ctrl
import pandas as pd

# Defining our endowments

In [2]:
original_task_endowments = ctrl.task_endowments
unique_task_endowments = list(set(ctrl.task_endowments))
unique_task_endowments.sort()
unique_task_endowments

[9, 15, 25, 40, 80, 125]

# Defining our progressivity levels

In [3]:
progressivity_systems = ctrl.progressivity_levels_tax_rates

# Calculating Public Incomes and Private Productivities

## Calculating Public Contributions

In [27]:
total_public_contributions = []

for system in progressivity_systems.values():
    current_public_contribution = 0 # public contrib for current tax rate
    
    counter = 0 # counter for tax rates
    for endowment in unique_task_endowments:
        tax_rate = system[counter]
        # calculating public contrib for current tax rate
        if endowment==15: # repeated 3 times
            current_public_contribution += 3*tax_rate*endowment 
        elif endowment==25: # repeated 2 times
            current_public_contribution += 2*tax_rate*endowment 
        else:
            current_public_contribution += tax_rate*endowment 
        
        counter += 1
        
    total_public_contributions.append(current_public_contribution) # storing current public contrib
    
total_public_contributions

[139.60000000000002, 138.57, 139.415, 138.765, 139.0]

## Processing Public Incomes and Private Productivities

In [28]:
public_incomes = []
private_productivities = []

for total_public_contribution in total_public_contributions:
    
    if total_public_contribution <= 192:
        # public income for current total contribution
        public_incomes.append(101 / (1 + 100 * math.exp(-0.025 * total_public_contribution)) - 1)
        # private productivity for current total contribution
        private_productivities.append(ctrl.alpha + ctrl.beta*total_public_contribution)
    else:
        public_incomes.append(101 / (1 + 100 * math.exp(-0.025 * 192)) - 1)
        private_productivities.append(ctrl.alpha + ctrl.beta*192)

In [29]:
public_incomes

[23.93773456853532,
 23.457296834138194,
 23.85097691877219,
 23.547768146849933,
 23.657099200195603]

In [30]:
private_productivities

[13.725000000000001, 13.660625, 13.7134375, 13.6728125, 13.6875]

# Calculating Private Incomes

In [40]:
private_incomes = {}

endowment_counter = 0 # counter unique endowments
for task_endowment in unique_task_endowments:
    private_incomes[f"private_incomes_{task_endowment}"] = [] # list with all the private incomes for a player
    
    tax_system = 1 # tax system counter
    for private_productivity in private_productivities: # calculating all the private incomes for an specific player
        tax_rate = progressivity_systems[f"{tax_system}"][endowment_counter] # tax rate in current system for current enwdowment
        private_incomes[f"private_incomes_{task_endowment}"].append(round(task_endowment*(1-tax_rate)*private_productivity, 3))
        tax_system += 1
        
    endowment_counter += 1

In [41]:
private_incomes

{'private_incomes_9': [74.115, 86.062, 98.737, 110.75, 123.188],
 'private_incomes_15': [123.525, 139.338, 156.333, 176.379, 205.312],
 'private_incomes_25': [205.875, 224.717, 245.471, 272.089, 321.656],
 'private_incomes_40': [329.4, 347.526, 368.617, 394.871, 403.507],
 'private_incomes_80': [658.8, 648.06, 615.459, 586.29, 513.555],
 'private_incomes_125': [1029.375, 925.507, 831.377, 709.277, 574.875]}

# Obtaining Final Payoffs

In [42]:
final_payoffs = {}

for task_endowment in unique_task_endowments:
    index = 0 # index for calling the private income/public contrib that corresponds to an specific tax rate
    
    final_payoffs[f"final_payoff_{task_endowment}"] = [] # list with all the final_payoffs for a player
    
    for tax_system in range(0,5): # calculating all the final payoffs for an specific player
        final_payoffs[f"final_payoff_{task_endowment}"].append(round(private_incomes[f"private_incomes_{task_endowment}"][tax_system] 
                                                              +public_incomes[tax_system], 3))
        index += 1

In [43]:
final_payoffs

{'final_payoff_9': [98.053, 109.519, 122.588, 134.298, 146.845],
 'final_payoff_15': [147.463, 162.795, 180.184, 199.927, 228.969],
 'final_payoff_25': [229.813, 248.174, 269.322, 295.637, 345.313],
 'final_payoff_40': [353.338, 370.983, 392.468, 418.419, 427.164],
 'final_payoff_80': [682.738, 671.517, 639.31, 609.838, 537.212],
 'final_payoff_125': [1053.313, 948.964, 855.228, 732.825, 598.532]}

# Creating our payoff table

In [44]:
payoffs_db = pd.DataFrame.from_dict(final_payoffs, orient='index')
payoffs_db.to_excel("payoffs_table_progressivity.xlsx")

D:\Anaconda5.2.0\lib\site-packages\pandas\compat\_optional.py:106: UserWarning: Pandas requires version '0.9.8' or newer of 'xlsxwriter' (version '0.9.3' currently installed).
  warnings.warn(msg, UserWarning)
